In [41]:
import os
import dicom
import nipy as ni
from nipy.core.api import AffineTransform as AfT, Image, vox2mni
import nibabel as nib
import numpy as np
import pandas as pd

In [42]:
# trainval = 'sample_images'
trainval = 'stage1'
# img_format = 'nii'
img_format = 'dcm'
path = os.path.join('../','data')
target_size = (512, 512, 128)

In [43]:
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

def iterative_mean(x_t, t, x_m, m):
    t_m = t + m
    x_t_m = x_t + (m / float(t+m)) * (x_m - x_t)
    return x_t_m, t_m

In [45]:
from nipy.labs.datasets.volumes.volume_img import VolumeImg
# load images
df_labels = pd.read_csv(path+"/dcm"+"/{}_labels.csv".format(trainval))

nb_series = len(df_labels)
# x = np.zeros((nb_series,)+target_size, dtype='float16')
xs = []

for idx, (s_id, series) in enumerate(zip(df_labels.index, df_labels['id'])):
    # load image
    if img_format == 'nii':
        # load
        img_path = os.path.join(path, img_format, trainval,
                                'Axial_{}.nii.gz'.format(series))
        img = ni.load_image(img_path)
        # resample
        vol_img = VolumeImg(data=img, affine=np.eye(4),
                            world_space='')
        resampled_img = vol_img.as_volume_img(affine=np.eye(4),
                                          shape=(512, 512, 128))

    elif img_format == 'dcm':
        # load
        img_path = os.path.join(path, img_format, trainval, series)
        patient = load_scan(img_path)
        patient_pixels = get_pixels_hu(patient)
        pix_resampled, spacing = resample(patient_pixels, patient, [1,1,1])

        # resample
        vol_img = VolumeImg(data=x_series, affine=np.eye(4),
                                world_space='')
        resampled_img = vol_img.as_volume_img(affine=np.eye(4),
                                          shape=(512, 512, 128))
        arr = resampled_img.get_data().astype(np.float64)
        # normalize
        arr -= float(np.mean(arr))
        arr /= float((np.amax(arr) - np.amin(arr)))
        # save
        img = Image(arr, vox2mni(np.eye(4)))
        output_path = os.path.join(path, 'norm_nii', trainval,
                                'Axial_{}.nii.gz'.format(series))
        newimg = ni.save_image(img, output_path)
    

df_labels['cancer'].values.astype('int8')

KeyboardInterrupt: 

In [40]:
print current_mean

5.06943572045e-17


In [31]:
print current_mean

161.074037505


In [26]:
print np.mean(xs), np.amin(xs), np.amax(xs)

507.43255497 0 2768
